## Ranking Model 학습시키기

### 1. csv 파일 준비하기 (모델 학습을 위한 데이터)

In [9]:
import mysql.connector as sql

import pandas as pd

host = '127.0.0.1'
port = 3306
username = 'root'
password = '1111'
database_name = 'dealicious'

db_connection = sql.connect(host=host, database=database_name, user=username, password=password)

retrievals_df = pd.read_sql('SELECT * FROM rec_retrievals', con=db_connection)

retrievals_df.head(3)

/var/folders/fx/b8s33_bs1nx_2nnz6h7w64qm0000gn/T/ipykernel_1737/2380683196.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  retrievals_df = pd.read_sql('SELECT * FROM rec_retrievals', con=db_connection)


,t_dat,customer_id,article_id,price,sales_channel_id,month_sin,month_cos,age,garment_group_name,index_group_name
0,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,710390001,0.019051,1,-0.866025,-0.5,20.0,Skirts,Divided
1,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,633130019,0.016932,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided
2,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,671057002,0.008458,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided


In [2]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(retrievals_df, test_size=0.2, random_state=42)

In [3]:
train_df["article_id"] = train_df["article_id"].astype(str) # to be deleted
test_df["article_id"] = test_df["article_id"].astype(str)

`USE_TRAIN` 를 각각 False, True 로 두고 아래의 코드 한번씩 실행
- ranking_train.csv
- ranking_test.csv 

In [23]:
USE_TRAIN = True

split, df = ("train", train_df) if USE_TRAIN else ("validation", test_df)
df['article_id'] = df['article_id'].astype(str)

ds_name = f"ranking_{split}.csv"

In [24]:
# positive_pairs : 실제 구매했던 (고객, 상품) 쌍
query_features = ["customer_id", "age", "month_sin", "month_cos"]

positive_pairs = df[query_features + ["article_id"]].copy()
positive_pairs

,customer_id,age,month_sin,month_cos,article_id
15415,9ab3d0b8dfdfafd08b8a23db28a893f748ffce9d269415...,51.0,-1.000000,-1.836970e-16,710899001
829,e425b828c8fedc7456dcec7b3a167e2eacdfee45a39cd8...,20.0,-0.866025,-5.000000e-01,570003013
14449,2dc0942a0e854456fd08dc03c065b6e0cee1f5499d7176...,47.0,-1.000000,-1.836970e-16,704148003
5727,35be9d9171c603488f94b78402f2e8b8b9f4244fa84341...,20.0,-0.866025,-5.000000e-01,673677003
5814,4ca7eafddf27a6159ee1eb842ea20057ae60dcdf6f583d...,29.0,-0.866025,-5.000000e-01,399223024
...,...,...,...,...,...
11284,b80df3e553cfa079e8f82b7d2e5fa2140492d9a5474768...,35.0,-1.000000,-1.836970e-16,519749036
11964,a1137ac03d00a3f98ac4997d4c1c7bf9dcdcc707b8baf2...,47.0,-1.000000,-1.836970e-16,658841001
5390,e8f0fd33c39b1c5558eef13401c55f80898bb8f8eca19d...,38.0,-0.866025,-5.000000e-01,537688010
860,e9975722496e741f7de041a4093b991e34ff5a7513c554...,46.0,-0.866025,-5.000000e-01,649551006


In [25]:
# negative_paris : 임의로 샘플링한 구매 x (고객, 상품) 쌍
n_neg = len(positive_pairs)*10

negative_pairs = positive_pairs[query_features]\
    .sample(n_neg, replace=True, random_state=1)\
    .reset_index(drop=True)

negative_pairs["article_id"] = positive_pairs["article_id"]\
    .sample(n_neg, replace=True, random_state=2).to_numpy()

negative_pairs

,customer_id,age,month_sin,month_cos,article_id
0,21b4053393d49a39f9c950935af68ae16545e59c925ad2...,64.0,-1.000000,-1.836970e-16,562761001
1,895576481a1095ad66ab3279483f4323724e9d53d9f089...,43.0,-0.866025,-5.000000e-01,380609008
2,a1ed419b99e97c261727c228a79ca370423d68195613c3...,53.0,-1.000000,-1.836970e-16,685383002
3,311198a56b7e9f98913c8a0080e51f6039374e80fb71e6...,22.0,-0.866025,-5.000000e-01,452851006
4,041f6c7b3be2bd8e8f34a007fee89e4db4e5844c5a0271...,23.0,-0.866025,-5.000000e-01,633130020
...,...,...,...,...,...
147835,2ce30f36fd18893c20ab9fbc9a4669ab7297d61f1e1eff...,42.0,-0.866025,-5.000000e-01,636455009
147836,c49107cfb46bdfb7233f704ba74e116cb37c49becdf04f...,30.0,-0.866025,-5.000000e-01,661162002
147837,db4e50337cb9db6d7face32c50dbd9dc85eab1f18f253d...,33.0,-1.000000,-1.836970e-16,438333016
147838,0b2c608f0e2fe96b66eb29ccf000e1885cbc654594e8d0...,28.0,-1.000000,-1.836970e-16,618387001


In [26]:
# labeling 하기 1 = 구매 o, 0 = 구매 x
import pandas as pd

# Add labels.
positive_pairs["label"] = 1
negative_pairs["label"] = 0

# Concatenate.
ranking_df = pd.concat([positive_pairs, negative_pairs], ignore_index=True)
ranking_df.head()

,customer_id,age,month_sin,month_cos,article_id,label
0,9ab3d0b8dfdfafd08b8a23db28a893f748ffce9d269415...,51.0,-1.000000,-1.836970e-16,710899001,1
1,e425b828c8fedc7456dcec7b3a167e2eacdfee45a39cd8...,20.0,-0.866025,-5.000000e-01,570003013,1
2,2dc0942a0e854456fd08dc03c065b6e0cee1f5499d7176...,47.0,-1.000000,-1.836970e-16,704148003,1
3,35be9d9171c603488f94b78402f2e8b8b9f4244fa84341...,20.0,-0.866025,-5.000000e-01,673677003,1
4,4ca7eafddf27a6159ee1eb842ea20057ae60dcdf6f583d...,29.0,-0.866025,-5.000000e-01,399223024,1


In [27]:
# Merge with item features.
item_df = pd.read_sql('SELECT * FROM rec_articles', con=db_connection)
item_df.drop_duplicates(subset="article_id", inplace=True)
ranking_df = ranking_df.merge(item_df, on="article_id")

ranking_df.head(3)


/var/folders/fx/b8s33_bs1nx_2nnz6h7w64qm0000gn/T/ipykernel_1737/456510631.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  item_df = pd.read_sql('SELECT * FROM rec_articles', con=db_connection)


,customer_id,age,month_sin,month_cos,article_id,label,product_code,prod_name,product_type_no,product_type_name,...,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name
0,9ab3d0b8dfdfafd08b8a23db28a893f748ffce9d269415...,51.0,-1.000000,-1.836970e-16,710899001,1,710899,Harper Turtleneck,252,Sweater,...,1636,Jersey fancy,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1005,Jersey Fancy
1,e425b828c8fedc7456dcec7b3a167e2eacdfee45a39cd8...,20.0,-0.866025,-5.000000e-01,570003013,1,570003,V-NECK SS SLIM FIT,255,T-shirt,...,5832,Light Basic Jersey,F,Menswear,3,Menswear,26,Men Underwear,1002,Jersey Basic
2,2dc0942a0e854456fd08dc03c065b6e0cee1f5499d7176...,47.0,-1.000000,-1.836970e-16,704148003,1,704148,Flirty Metal Stripe Phone case,78,Other accessories,...,3419,Other items,C,Ladies Accessories,1,Ladieswear,66,Womens Small accessories,1019,Accessories


In [28]:
def exclude_feat(s):
    return s.endswith("_id") or s.endswith("_no") or s.endswith("_code")

features_to_exclude = [col for col in ranking_df.columns if exclude_feat(col)]

ranking_df.drop(features_to_exclude, axis="columns", inplace=True)
ranking_df.head()

,age,month_sin,month_cos,label,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name
0,51.0,-1.000000,-1.836970e-16,1,Harper Turtleneck,Sweater,Garment Upper body,Melange,Yellow,Medium Dusty,Yellow,Jersey fancy,Ladieswear,Ladieswear,Womens Everyday Collection,Jersey Fancy
1,20.0,-0.866025,-5.000000e-01,1,V-NECK SS SLIM FIT,T-shirt,Garment Upper body,Stripe,Dark Red,Dark,Red,Light Basic Jersey,Menswear,Menswear,Men Underwear,Jersey Basic
2,47.0,-1.000000,-1.836970e-16,1,Flirty Metal Stripe Phone case,Other accessories,Accessories,All over pattern,Dark Green,Dark,Green,Other items,Ladies Accessories,Ladieswear,Womens Small accessories,Accessories
3,20.0,-0.866025,-5.000000e-01,1,Henry polo (1),Sweater,Garment Upper body,Solid,Dark Blue,Dark,Blue,Knitwear,Ladieswear,Ladieswear,Womens Tailoring,Knitwear
4,29.0,-0.866025,-5.000000e-01,1,Jeggings H.W,Trousers,Garment Lower body,Denim,Blue,Medium Dusty,Blue,Denim Trousers,Divided,Divided,Ladies Denim,Trousers Denim


In [29]:
ranking_df.to_csv(ds_name, index=False)